**<center><font size = "6"> 4. División de Datos <center>**
***

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Regresión" data-toc-modified-id="Regresión-1">Regresión</a></span></li><li><span><a href="#Clasificación" data-toc-modified-id="Clasificación-2">Clasificación</a></span></li><li><span><a href="#Almacenamiento-de-features" data-toc-modified-id="Almacenamiento-de-features-3">Almacenamiento de features</a></span></li></ul></div>

***

In [1]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import EditedNearestNeighbours
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import pickle
import plotly.express as px


data_text_tfidf = pickle.load(open("saved_feats/data_text_tfidf", "rb"))
data_sentiment = pickle.load(open("saved_feats/data_sentiment", "rb"))
data_obj_r = pickle.load(open("saved_feats/data_obj_r", "rb"))
data_obj_c = pickle.load(open("saved_feats/data_obj_c", "rb"))

# Regresión

In [2]:
# Unir data_text_tfidf y data_sentiment en un solo array
X = np.hstack((data_text_tfidf.toarray(), data_sentiment.reshape(-1, 1)))

In [3]:
print(X.shape)

(13442, 12351)


In [4]:
print(data_obj_r.shape)

(13442,)


In [5]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(
    X, data_obj_r, test_size=0.2, random_state=42)

# Clasificación

In [6]:
uniques, counts = np.unique(data_obj_c, return_counts=True)
percentages = dict(zip(uniques, counts * 100 / len(data_sentiment)))
percentages

{'Baja': 1.763130486534742,
 'Estable': 96.36214848980806,
 'Sube': 1.8747210236571938}

In [7]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(
    X, data_obj_c, test_size=0.2, random_state=42)

In [8]:
# Calcular la estrategia de muestreo para SMOTE
strategy = {
    'Baja': 15000,     
    'Estable': 25000, 
    'Sube': 15000      
}

# Aplicar SMOTE con estrategia personalizada solo en los datos de entrenamiento
smote = SMOTE(sampling_strategy=strategy, random_state=42)
X_train_smt, y_train_smt = smote.fit_resample(X_train_c, y_train_c)

# Aplicar Edited Nearest Neighbours para eliminar muestras ruidosas
enn = EditedNearestNeighbours(n_neighbors=3)
X_train_smt, y_train_smt = enn.fit_resample(X_train_smt, y_train_smt)

In [16]:
# Crear un DataFrame con los datos de entrenamiento balanceados
data_train_smt = pd.DataFrame({'Sentimiento': y_train_smt})

# Crear el histograma con etiquetas
fig = px.histogram(data_train_smt, x='Sentimiento',
                   labels={"Sentimiento": "Sentimiento"})

# Agregar etiquetas a las barras del histograma
fig.update_traces(texttemplate='%{y}', textposition='outside')

fig.update_layout(
    title_text='Histograma de Sentimientos Balanceados',
    xaxis_title_text='Sentimiento',
    yaxis_title_text='Frecuencia',
)

fig.show()

# Almacenamiento de features

In [10]:
pickle.dump(X_train_r, open("saved_feats/X_train_r", "wb"))
pickle.dump(X_test_r, open("saved_feats/X_test_r", "wb"))
pickle.dump(y_train_r, open("saved_feats/y_train_r", "wb"))
pickle.dump(y_test_r, open("saved_feats/y_test_r", "wb"))

pickle.dump(X_train_smt, open("saved_feats/X_train_smt", "wb"))
pickle.dump(X_test_c, open("saved_feats/X_test_c", "wb"))
pickle.dump(y_train_smt, open("saved_feats/y_train_smt", "wb"))
pickle.dump(y_test_c, open("saved_feats/y_test_c", "wb"))